In [ ]:
# Imports to fine-tune BERT-like text classification model using transformers

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np



In [ ]:
# For this datset and the scientific domain, there is a specialized BERT model fine-tuned for scientific texts
# and presented by Beltagy et al. - SCIBERT: A Pretrained Language Model for Scientific Text - https://aclanthology.org/D19-1371.pdf
# model_name = "allenai/scibert_scivocab_uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# Uncomment to use this model instead of distilBERT. Unsufficient RAM and CPU resources on the current setup for this model.



In [ ]:
hf_datasets = load_dataset('csv', data_files={
    'train': 'arxiv_200_multiclass_train.csv',
    'validation': 'arxiv_200_multiclass_val.csv',
    'test': 'arxiv_200_multiclass_test.csv'
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

In [ ]:
tokenized_train = hf_datasets['train'].map(preprocess_function, batched=True)
tokenized_val = hf_datasets['validation'].map(preprocess_function, batched=True)
tokenized_test = hf_datasets['test'].map(preprocess_function, batched=True)

In [ ]:
list_of_labels = ['hep-ph',
 'physics',
 'math',
 'cond-mat',
 'gr-qc',
 'astro-ph',
 'hep-th',
 'hep-ex',
 'nlin',
 'q-bio',
 'cs',
 'nucl-th',
 'quant-ph',
 'nucl-ex',
 'hep-lat',
 'stat',
 'q-fin',
 'eess',
 'econ']
id2label = {i: label for i, label in enumerate(list_of_labels)}
label2id = {label: i for i, label in enumerate(list_of_labels)}

In [ ]:

# Load the base model

model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-uncased", num_labels=len(list_of_labels), label2id=label2id, id2label=id2label)

# TODO: Test SciBERT model as well

In [ ]:


accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_multiclass",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    #compute_metrics=compute_metrics, This is not working here. Tried to change transformers versions but no success. Will compute them separately after training in an evalution notebook on the test set.
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    use_cpu=True,
    load_best_model_at_end=True,
)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    #tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()